In [1]:
import pandas as pd
import numpy as np
import locale
from locale import atof
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

## Population classification by sex

In [2]:
population = pd.read_csv('Population.csv')
population_total_over_age = population[population['AGE']=='Total']    #Consider the total over all ages
male_pop = population_total_over_age[population_total_over_age['SEX']=='Males']    #Classify into male and female population
female_pop = population_total_over_age[population_total_over_age['SEX']=='Females']
male_pop = male_pop[['TIME','GEO','Value']]                                    #Select data only for time, region and population    
female_pop = female_pop[['TIME','GEO','Value']]   
male_pop.columns = ['TIME','GEO','males']                                     #Rename columns
female_pop.columns = ['TIME','GEO','females']
population_sex = male_pop.merge(female_pop,on=['TIME','GEO'])                #Merge male and female data as separate columns
population_sex = population_sex.replace({',':''},regex=True)                 #Convert population to numeric format
population_sex.replace({":":0}, inplace=True)                                 
for column in population_sex.columns:
    population_sex[column] = pd.to_numeric(population_sex[column],errors='ignore')
#population_age.applymap(atof)

## Population classification by age and income

In [3]:
population_age = pd.DataFrame()
population_age['TIME'] = population_sex['TIME']
population_age['GEO'] = population_sex['GEO']
population_total_over_sex = population[population['SEX']=='Total']           #Get data over both male and female
for age in population_total_over_sex['AGE'].unique():                        #integrate age divided data with the dataframe  
    age_group = population_total_over_sex[population_total_over_sex['AGE']==age]
    age_group = age_group[['TIME','GEO','Value']]
    age_group.columns = ['TIME','GEO','number_ ' + age]
    population_age = population_age.merge(age_group,on=['TIME','GEO'])
population_age = population_age.replace({',':''},regex=True)               #Convert non numeric data to numeric format       
population_age.replace({":":0}, inplace=True) 
for column in population_age.columns:                               
    population_age[column] = pd.to_numeric(population_age[column],errors='ignore')
#population_age.applymap(atof)
#Divide age cohorts into ages for which data is avaialble in Hague cijfers
population_age['0 to 15 year'] =  population_age['number_ Less than 5 years'] + population_age['number_ From 5 to 9 years']+ population_age['number_ From 10 to 14 years']
population_age['15 to 25 year'] = population_age['number_ From 15 to 19 years'] + population_age['number_ From 20 to 24 years']
population_age['25 to 45 year'] = population_age['number_ From 25 to 29 years'] + population_age['number_ From 30 to 34 years'] \
                                  + population_age['number_ From 35 to 39 years'] + population_age['number_ From 40 to 44 years']
population_age['45 to 65 year'] = population_age['number_ From 45 to 49 years'] + population_age['number_ From 50 to 54 years'] \
                                  + population_age['number_ From 55 to 59 years'] + population_age['number_ From 60 to 64 years']
population_age['65 or older'] = population_age['number_ Total'] - population_age['0 to 15 year'] - population_age['15 to 25 year']\
                                - population_age['25 to 45 year'] - population_age['45 to 65 year']
population_age = population_age[['TIME','GEO','0 to 15 year','15 to 25 year', '25 to 45 year', '45 to 65 year', '65 or older']]
input_data = population_sex.merge(population_age,how='inner',on=['TIME','GEO'])   #Integrate age data with sex data
income = pd.read_csv('Income.csv')
input_data = pd.merge(input_data,income,how='inner',on=['TIME','GEO'])          #Integrate income data with existing database

## Adding education data for different countries

In [4]:
edu = pd.read_csv('Education.csv')
i = 0
for education in edu['ISCED11'].unique():
    if (i == 0):
        i = i + 1
        education_categorized = edu[edu['ISCED11']== education]
        education_categorized = education_categorized[['TIME','GEO','Value']]
        education_categorized.columns =['TIME','GEO',education]
    else :
        category = edu[edu['ISCED11']== education]
        category = category[['TIME','GEO','Value']]
        category.columns = ['TIME','GEO',education]
        education_categorized = education_categorized.merge(category,on=['TIME','GEO'],how='inner')
education_categorized = education_categorized.iloc[:,[0,1,2,4,5]]
education_categorized.columns=['TIME','GEO','Primary','Secondary','Tertiary']
education_categorized['Primary'] = pd.to_numeric(education_categorized['Primary'],errors='coerce')
education_categorized['Secondary'] = pd.to_numeric(education_categorized['Secondary'],errors='coerce')
education_categorized['Tertiary'] = pd.to_numeric(education_categorized['Tertiary'],errors='coerce')
input_data = pd.merge(input_data,education_categorized,how='inner',on=['TIME','GEO'])

## Adding employment data for different countries

In [5]:
employ = pd.read_csv('ActivityLevel.csv')
employ = employ[employ['SEX']=='Total']
employ['time and country'] = employ['TIME'].str[0:4] + employ['GEO']
employ.replace({":":np.NaN}, inplace=True)
employ = employ[['time and country','Value']]
employ['Value'] = pd.to_numeric(employ['Value'],errors = 'ignore')
employ_new = employ.groupby('time and country').mean()
employ_new['time and country']=employ_new.index
employ_new['TIME'] = employ_new['time and country'].str[0:4]
employ_new['GEO'] = employ_new['time and country'].str[4:]
employ_new = employ_new[['TIME','GEO','Value']]
employ_new.columns = ['TIME','GEO','activity_level']
employ_new = employ_new.reset_index(drop=True)
employ_new['TIME'] = pd.to_numeric(employ_new['TIME'],errors = 'ignore')
input_data = pd.merge(input_data,employ_new,how='inner',on=['TIME','GEO'])
input_data = input_data[input_data['UNIT']=='Euro']
input_data.drop(['HHTYP','INDIC_IL','UNIT','Flag and Footnotes'], axis=1, inplace=True)


## Modifying the input data to percentages

In [6]:
input_data['Total_pop'] = input_data['males'] + input_data['females']
input_data['male_percent'] = input_data['males']/input_data['Total_pop']
input_data['female_percent'] = input_data['females']/input_data['Total_pop']
input_data['0 to 15 percent'] = input_data['0 to 15 year']/input_data['Total_pop']
input_data['15 to 25 percent'] = input_data['15 to 25 year']/input_data['Total_pop']
input_data['25 to 45 percent'] = input_data['25 to 45 year']/input_data['Total_pop']
input_data['45 to 65 percent'] = input_data['45 to 65 year']/input_data['Total_pop']
input_data['65 or older percent'] = input_data['65 or older']/input_data['Total_pop']
input_data['Primary'] = input_data['Primary']*0.01 
input_data['Secondary'] = input_data['Secondary']*0.01 
input_data['Tertiary'] = input_data['Tertiary']*0.01 
input_data['activity_level'] = input_data['activity_level']*0.01
input_data['Value']=input_data['Value'].replace({",":''}, regex=True)
input_data['Value'] = pd.to_numeric(input_data['Value'],errors='coerce')
input_data.rename(columns = {'Value':'Income'}, inplace = True)
final_input_data = input_data[['TIME','GEO','male_percent','female_percent','0 to 15 percent','15 to 25 percent',
                              '25 to 45 percent','45 to 65 percent','65 or older percent','Primary','Secondary',
                              'Tertiary','activity_level','Income','Total_pop']]
final_input_data.to_csv('inputdata.csv',index=False)